In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ETL Dataset

In [ ]:
df=pd.read_excel("/kaggle/input/sales-ecommerce/2_Ecommerce Sales Data Analysis Excel.xlsx")

In [ ]:
df.head()

# Recency

In [ ]:
df["Date"]=pd.to_datetime(df["Order Date"])
df["rank"]=df.sort_values(["Order ID","Date"]).groupby(["Customer ID"])["Date"].rank(method='min').astype(int)
df_rec=df[df["rank"]==1]

In [ ]:
df_rec["recency"]=(df_rec["Date"] - pd.to_datetime(min(df_rec["Date"]))).dt.days

# frequency

In [ ]:
freq=df_rec.groupby('Customer ID')["Date"].count()
df_freq=pd.DataFrame(freq).reset_index()
df_freq.columns=['Customer ID','frequency']

In [ ]:
rec_freq=df_freq.merge(df_rec,on="Customer ID")

In [ ]:
rec_freq

In [ ]:
def calculate_total(row):
    if row["Discount"]==0:
        return row["Quantity"]*row["Sales"] 
    elif row["Discount"]%row["Sales"]!=0 or row["Discount"]%row["Quantity"]!=0:
        return row["Quantity"]*row["Sales"] / row["Discount"]
    else:
        return "error"
rec_freq["total"]=rec_freq.apply(calculate_total,axis=1)
m=rec_freq.groupby("Customer ID")["total"].sum()
m=pd.DataFrame(m).reset_index()
m.columns=["Customer ID","monetary_values"]

# calculate total and monetary value

In [ ]:
rfm=m.merge(rec_freq,on="Customer ID")

In [ ]:
finaldf=rfm[["Customer ID","recency","frequency","monetary_values"]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
list=["recency","frequency","monetary_values"]
for i in list:
    print(str(i)+": ")
    ax=sns.boxplot(x=finaldf[str(i)])
    plt.show()

# remove outliers

In [ ]:
from scipy import stats
new_df=finaldf[["recency","frequency","monetary_values"]]
z_score=stats.zscore(new_df)
abs_z=np.abs(z_score)
filtered=(abs_z<4).all(axis=1)
new_df=new_df[filtered]

In [ ]:
new_df

In [ ]:
from sklearn.preprocessing import StandardScaler
new_df=new_df.drop_duplicates()
col_names=["recency","frequency","monetary_values"]
features=new_df[col_names]
scaler=StandardScaler().fit(features.values)
features=scaler.transform(features.values)
scaled_f=pd.DataFrame(features,columns=col_names)

# building customer segment model

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
SSE=[]
for cluster in range(1,8):
    kmeans=KMeans(n_clusters=cluster,init='k-means++')
    kmeans.fit(scaled_f)
    SSE.append(kmeans.inertia_)
frame=pd.DataFrame({'cluster':range(1,8),'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['cluster'],frame['SSE'],marker='o')
plt.xlabel("num of cluster")
plt.ylabel("inertia")

In [ ]:
kmeans=KMeans(n_clusters=4,init='k-means++')
kmeans.fit(scaled_f)

In [ ]:
print(silhouette_score(scaled_f,kmeans.labels_,metric='euclidean'))

In [ ]:
predict=kmeans.predict(scaled_f)
frame=pd.DataFrame(new_df)
frame['cluster']=predict

In [ ]:
frame

In [ ]:
avg_df=frame.groupby(['cluster'],as_index=False).mean()
for i in list:
    sns.barplot(x='cluster',y=str(i),data=avg_df)
    plt.show()